# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,anadyr,64.7500,177.4833,3.28,86,75,4.92,RU,1694280809
1,1,utrik,11.2278,169.8474,28.17,77,59,5.55,MH,1694280809
2,2,balsas,-7.5325,-46.0356,38.78,18,48,2.66,BR,1694280809
3,3,berezovyy,51.6667,135.7000,7.12,53,23,2.34,RU,1694280810
4,4,margaret river,-33.9500,115.0667,15.54,64,100,6.83,AU,1694280810


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
                               ]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity: {longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel", "Country"]
)

# Display the map
map_plot